##### The purpose of this notebook is to do some 'pre-cleaning' of the data, so we have a training and test set that is ready to be further cleaned and analyzed.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import presetup
import seaborn as sns
import re
from collections import Counter
from sklearn.cross_validation import train_test_split
%matplotlib inline

In [3]:
df = pd.read_csv('../data/raw/raw_data.csv', low_memory=False)

In [4]:
print df.columns.values

['entry_id' 'field_id_4' 'field_id_8' 'field_id_13' 'field_id_15'
 'field_id_17' 'field_id_18' 'field_id_19' 'field_id_20' 'field_id_21'
 'field_id_23' 'field_id_25' 'field_id_27' 'field_id_28' 'field_id_29'
 'field_id_33' 'field_id_40' 'field_id_41' 'field_id_42' 'field_id_68'
 'field_id_70' 'field_id_71' 'field_id_72' 'field_id_73' 'field_id_74'
 'field_id_75' 'field_id_89' 'field_id_96' 'field_id_98' 'field_id_100'
 'field_id_101' 'field_id_104' 'field_id_105' 'field_id_106' 'field_id_107'
 'field_id_108' 'field_id_111' 'field_id_112' 'field_id_113' 'field_id_114'
 'field_id_115' 'field_id_116' 'field_id_117' 'field_id_118' 'field_id_119'
 'field_id_120' 'field_id_121' 'field_id_122' 'field_id_123' 'field_id_124'
 'field_id_125' 'field_id_126' 'field_id_127' 'field_id_176' 'field_id_250'
 'field_id_252' 'field_id_264' 'field_id_329' 'field_id_330' 'field_id_331'
 'field_id_394' 'field_id_395' 'field_id_410' 'field_id_411' 'field_id_412'
 'field_id_413' 'field_id_414' 'field_id_426' 

Let's instantiate an instance of the PreSetup class from our custom module 'presetup' - we'll use it to fetch a list of interpretable column names.

In [8]:
ps = presetup.PreSetup()
col_dict = ps.parseCols('../data/reference/column_names.txt')
df.columns = ps.updateCols(df.columns.values)

In [9]:
print df.columns.values

['id' 'Gender' 'Undergraduate Schools Attended'
 'Undergraduate Graduation Year' 'Undergraduate Major(s)' 'Ethnicity'
 'Home State or Territory (US)' 'Home City' 'High School Type'
 'High School GPA' 'High School Class Rank'
 'High School AP / IB Classes and Scores' 'Undergraduate Additional Info'
 'High School Extracurricular Activities' 'High School Sports Played'
 'High School Other Honors and Awards' 'Undergraduate Acceptance'
 'Undergraduate Month Applied' 'Undergraduate Schools Applied'
 'Who are you?' 'About Me' 'Birth Year' 'Home Country'
 'Is English your first language?' 'Do you speak other languages?'
 'Undergrad Legacy' 'Undergraduate Degree Type'
 'Undergraduate Type of Major(s)' 'Highest SAT Scores'
 'How many times did you take the official SAT?'
 'SAT 2 Tests Taken and Highest Scores'
 'Undergraduate Personal Statement Topic'
 'Undergraduate Personal Statement Type' 'Undergraduate Personal Statement'
 'Undergraduate Supplemental Essay'
 'How did you spend your summers d

In [11]:
df['Who are you?'].value_counts()

Admit Creating College / Grad School Profile    18465
Student Applying To College / Grad School         102
Other Professional                                 32
School Faculty / Professional                      12
Parent                                              7
Name: Who are you?, dtype: int64

We're only interested in the profiles of those who have gone through the college admissions process and are not school faculty or parents.

In [12]:
df = df[df['Who are you?']=='Admit Creating College / Grad School Profile'].copy()
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

Some more housekeeping: let's replace the base64 nulls.

In [13]:
vals = ['YTowOnt9', 'ytowont9', 'czowOiIiOw==']
for v in vals:
    df.replace(to_replace=v, value=np.nan, inplace=True)

### The Target Variable: Acceptance into a Top School

Steps:
1. Instantiate a new instance of the Schools class.
2. Get list of all schools.
3. Create a new DataFrame df_schools, where cols are schools, rows are students
4. Update acceptance status for each school in df_schools.
5. Create a separate DataFrame df_topschools (where the cols are just the 'top schools').
6. Convert each col into binary (1 for accepted, 0 for not), then create new col called any_top_school to indicate acceptance into any top school.
7. Join back with main df, and make any necessary adjustments.

#### Step 1
Instantiate a new instance of the Schools class.

In [14]:
sc = presetup.Schools()

#### Step 2
Get list of all schools.

In [17]:
all_schools = sc.getSchools('../data/reference/table_references.csv')

In [18]:
print len(all_schools), len(set(all_schools))

1355 1353


There are a few duplicates- let's remove these.

In [19]:
all_schools = list(set(all_schools))

#### Step 3
Create a new DataFrame df_schools, where cols are schools, rows are students

In [20]:
df_schools = pd.DataFrame(index=xrange(len(df)), columns=all_schools)

#### Step 4
Update acceptance status for each school in df_schools.

In [21]:
sc.extractFromApplied(df['Undergraduate Schools Applied'], df_schools)

Due to some formatting hiccups in the data, a handful of non-school columns were generated. Let's get rid of these.

In [22]:
df_schools = df_schools[all_schools]

#### Step 5
Create a separate DataFrame df_topschools (where the cols are just the 'top schools').

In [23]:
top_schools = ['Harvard University (Cambridge, MA)', 'Yale University (New Haven, CT)', 
               'Cornell University (Ithaca, NY)', 'Columbia University (New York, NY)',
               'University of Pennsylvania (Philadelphia, PA)', 'Princeton University (Princeton, NJ)',
               'Brown University (Providence, RI)', 'Dartmouth College (Hanover, NH)',
               'Massachusetts Institute of Technology (Cambridge, MA)','Stanford University (Stanford, CA)']
df_topschools = df_schools[top_schools].copy()

#### Step 6
Convert each col into binary (1 for accepted, 0 for not), then create new col called any_top_school to indicate acceptance into any top school.

In [24]:
for school in top_schools:
    df_topschools[school] = df_topschools[school].apply(lambda x: sc.cleanFromApplied(x) if type(x) == str else x)

In [25]:
df_topschools['any_top_school'] = (df_topschools.sum(axis=1)).apply(lambda x: 1 if x>0 else np.nan)

#### Step 7
Join back with main df, and make any necessary adjustments

In [26]:
# Join df_topschools back with main df
df = df.join(df_topschools)

In [27]:
print df['Undergraduate Schools Applied'].notnull().sum()
print df['Undergraduate Schools Attended'].notnull().sum()

6614
17639


There's another problem here- we just extracted info out of the 'Undergraduate Schools Applied' column, but it has far more null values than 'Undergraduate Schools Attended', which means we're currently missing out on a lot of potentially important data. Let's cover our bases by incorporating the data in the Attended column.

In [28]:
df['any_top_school_v2'] =  df['Undergraduate Schools Attended'].apply(sc.extractFromAttended)

Now let's combine the data from the two columns to get top_school_final.

In [29]:
df['top_school_final'] = df.apply(sc.finalTopSchool, axis=1)

Note that in that process, we have made an implicit assumption that those who did not apply to a top school would not have gotten in anyway. While it may not be the most robust approach, we assume that students that actually have a fair chance of getting into a top school would have applied anyway, and that result is reflected in our data.

### Remove Rows that are Too Sparse

Over-sparseness is a huge problem in this data set. Fortunately, AdmitSee has already internally created a function to determine the 'usefulness' (how much is filled) of each profile, indicated by the 'Internal Use - Calculated Undergrad Price'. As suggested by their Founding Engineer, let's only keep those > 4 for this column.

In [30]:
df2 = df[df['Internal Use - Calculated Undergrad Price']>4].copy()

In [31]:
target_distr = df2['top_school_final'].value_counts()
print target_distr

0    4416
1    1657
Name: top_school_final, dtype: int64


In [32]:
print target_distr/float(sum(target_distr))

0    0.727153
1    0.272847
Name: top_school_final, dtype: float64


Great- our task is now done. We have about a quarter of students who got into a top school.

### Train/Test Split

As a final step in this pre-setup, let's do a 70-30 train test split, and export the data.

In [33]:
df_train, df_test = train_test_split(df, train_size=0.7, random_state=123)

In [34]:
df_train.to_csv('../data/train.csv')
df_test.to_csv('../data/test.csv')
df.to_csv('../data/master.csv')